In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf
from tensorflow.python.saved_model import signature_constants
import mlflow
import numpy as np
img_width, img_height = 224, 224

In [2]:
train_data_dir = 'v_data/train'
validation_data_dir = 'v_data/test'
nb_train_samples =400
nb_validation_samples = 100
epochs = 20
batch_size = 16

In [3]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
train_datagen = ImageDataGenerator(rescale=1. / 255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,target_size=(img_width, img_height),batch_size=batch_size,
class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 400 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [5]:
saved_model_path=r"Model"
saved_model_path1=r"Model1"
saved_model_path2=r"Model2"

In [6]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
reg_model_name = "First_Model"
mlflow.set_tracking_uri("http://localhost:5000")
tag=[tf.compat.v1.saved_model.tag_constants.SERVING]
key=signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY

In [7]:
with mlflow.start_run():
    model = Sequential()
    model.add(Conv2D(32, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history=model.fit(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
    tf.keras.models.save_model(model,saved_model_path)
    mlflow.tensorflow.log_model(tf_saved_model_dir=saved_model_path,
                                tf_meta_graph_tags=tag,
                                tf_signature_def_key=key,
                                artifact_path=saved_model_path,
                                registered_model_name=reg_model_name)
    for i in history.history:
        history.history[i]=history.history[i][-1]
    mlflow.log_metrics(history.history)

Epoch 1/20
25/25 [==============================] - 8s 268ms/step - loss: 0.7034 - accuracy: 0.6400 - val_loss: 0.5549 - val_accuracy: 0.8750
Epoch 2/20
25/25 [==============================] - 7s 287ms/step - loss: 0.5299 - accuracy: 0.7500 - val_loss: 0.4509 - val_accuracy: 0.8646
Epoch 3/20
25/25 [==============================] - 8s 309ms/step - loss: 0.4494 - accuracy: 0.7950 - val_loss: 0.3825 - val_accuracy: 0.8854
Epoch 4/20
25/25 [==============================] - 7s 276ms/step - loss: 0.3984 - accuracy: 0.8450 - val_loss: 0.3221 - val_accuracy: 0.8750
Epoch 5/20
25/25 [==============================] - 7s 285ms/step - loss: 0.3286 - accuracy: 0.8775 - val_loss: 0.2817 - val_accuracy: 0.8958
Epoch 6/20
25/25 [==============================] - 7s 267ms/step - loss: 0.3140 - accuracy: 0.8625 - val_loss: 0.2863 - val_accuracy: 0.8854
Epoch 7/20
25/25 [==============================] - 7s 271ms/step - loss: 0.2749 - accuracy: 0.8700 - val_loss: 0.3062 - val_accuracy: 0.8750
Epoch 

INFO:tensorflow:Assets written to: Model\assets


INFO:tensorflow:Assets written to: Model\assets
2022/10/19 20:39:32 INFO mlflow.tensorflow: Validating the specified TensorFlow model by attempting to load it in a new TensorFlow graph...
2022/10/19 20:39:33 INFO mlflow.tensorflow: Validation succeeded!
C:\Users\med-a\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'First_Model'.
2022/10/19 20:39:42 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: First_Model, version 1
Created version '1' of model 'First_Model'.


In [8]:
from tensorflow.keras.utils import load_img
image = load_img('v_data/test/planes/3.jpg', target_size=(224, 224))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = model.predict(img)
if (label[0][0]<0.5) and (label[0][1]<0.5) :
    print("This is NOT a car NOR a plane")
elif label[0][0]<label[0][1]  :
    print("Predicted Class Planes")
else :
    print("Predicted Class Cars")

1/1 [==============================] - 0s 125ms/step
Predicted Class Planes


In [9]:
reg_model_name1="Second Model"
with mlflow.start_run():
    model1 = Sequential()
    model1.add(Conv2D(32, (2, 2)))
    model1.add(Activation('relu'))
    model1.add(MaxPooling2D(pool_size=(2, 2)))
    model1.add(Dropout(0.2))
    model1.add(Flatten())
    model1.add(Dense(64))
    model1.add(Activation('relu'))
    model1.add(Dense(32))
    model1.add(Activation('relu'))
    model1.add(Dropout(0.2))
    model1.add(Dense(2))
    model1.add(Activation('softmax'))
    model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history=model1.fit(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
    tf.keras.models.save_model(model,saved_model_path1)
    mlflow.tensorflow.log_model(tf_saved_model_dir=saved_model_path1,
                                tf_meta_graph_tags=tag,
                                tf_signature_def_key=key,
                                artifact_path=saved_model_path1,
                                registered_model_name=reg_model_name1)
    for i in history.history:
        history.history[i]=history.history[i][-1]
    mlflow.log_metrics(history.history)

Epoch 1/20
25/25 [==============================] - 8s 297ms/step - loss: 1.9404 - accuracy: 0.5900 - val_loss: 0.4070 - val_accuracy: 0.8229
Epoch 2/20
25/25 [==============================] - 7s 279ms/step - loss: 0.6171 - accuracy: 0.7125 - val_loss: 0.2425 - val_accuracy: 0.8750
Epoch 3/20
25/25 [==============================] - 7s 294ms/step - loss: 0.4147 - accuracy: 0.7875 - val_loss: 0.1937 - val_accuracy: 0.9375
Epoch 4/20
25/25 [==============================] - 7s 287ms/step - loss: 0.3495 - accuracy: 0.8425 - val_loss: 0.2208 - val_accuracy: 0.9167
Epoch 5/20
25/25 [==============================] - 7s 291ms/step - loss: 0.3296 - accuracy: 0.8600 - val_loss: 0.2484 - val_accuracy: 0.9271
Epoch 6/20
25/25 [==============================] - 7s 295ms/step - loss: 0.3334 - accuracy: 0.8625 - val_loss: 0.3066 - val_accuracy: 0.8542
Epoch 7/20
25/25 [==============================] - 8s 311ms/step - loss: 0.2589 - accuracy: 0.8975 - val_loss: 0.2717 - val_accuracy: 0.8854
Epoch 

INFO:tensorflow:Assets written to: Model1\assets


INFO:tensorflow:Assets written to: Model1\assets
2022/10/19 20:42:15 INFO mlflow.tensorflow: Validating the specified TensorFlow model by attempting to load it in a new TensorFlow graph...
2022/10/19 20:42:16 INFO mlflow.tensorflow: Validation succeeded!
Registered model 'Second Model' already exists. Creating a new version of this model...
2022/10/19 20:42:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Second Model, version 2
Created version '2' of model 'Second Model'.


In [10]:
reg_model_name2="Third Model"
with mlflow.start_run():
    model2 = Sequential()
    model2.add(Conv2D(64, (2, 2), input_shape=input_shape))
    model2.add(Activation('relu'))
    model2.add(Dropout(0.5))
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Conv2D(32, (2, 2), input_shape=input_shape))
    model2.add(Activation('relu'))
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Dropout(0.2))
    model2.add(Conv2D(32, (2, 2)))
    model2.add(Activation('relu'))
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Dropout(0.2))
    model2.add(Conv2D(64, (2, 2)))
    model2.add(Activation('relu'))
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Dropout(0.2))
    model2.add(Flatten())
    model2.add(Dense(64))
    model2.add(Activation('relu'))
    model2.add(Dense(32))
    model2.add(Activation('relu'))
    model2.add(Dropout(0.2))
    model2.add(Dense(2))
    model2.add(Activation('softmax'))
    model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history=model2.fit(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
    tf.keras.models.save_model(model2,saved_model_path2)
    mlflow.tensorflow.log_model(tf_saved_model_dir=saved_model_path2,
                                tf_meta_graph_tags=tag,
                                tf_signature_def_key=key,
                                artifact_path=saved_model_path2,
                                registered_model_name=reg_model_name2)
    for i in history.history:
        history.history[i]=history.history[i][-1]
    mlflow.log_metrics(history.history)

Epoch 1/20
25/25 [==============================] - 19s 705ms/step - loss: 0.7310 - accuracy: 0.5050 - val_loss: 0.6927 - val_accuracy: 0.4896
Epoch 2/20
25/25 [==============================] - 18s 729ms/step - loss: 0.6320 - accuracy: 0.6750 - val_loss: 0.6059 - val_accuracy: 0.5833
Epoch 3/20
25/25 [==============================] - 19s 761ms/step - loss: 0.5662 - accuracy: 0.7250 - val_loss: 0.5484 - val_accuracy: 0.6458
Epoch 4/20
25/25 [==============================] - 18s 731ms/step - loss: 0.4285 - accuracy: 0.7975 - val_loss: 0.4751 - val_accuracy: 0.8229
Epoch 5/20
25/25 [==============================] - 19s 741ms/step - loss: 0.3437 - accuracy: 0.8450 - val_loss: 0.4795 - val_accuracy: 0.8021
Epoch 6/20
25/25 [==============================] - 18s 713ms/step - loss: 0.3287 - accuracy: 0.8550 - val_loss: 0.4531 - val_accuracy: 0.8229
Epoch 7/20
25/25 [==============================] - 18s 704ms/step - loss: 0.3279 - accuracy: 0.8600 - val_loss: 0.4276 - val_accuracy: 0.8125

INFO:tensorflow:Assets written to: Model2\assets


INFO:tensorflow:Assets written to: Model2\assets
2022/10/19 20:48:21 INFO mlflow.tensorflow: Validating the specified TensorFlow model by attempting to load it in a new TensorFlow graph...
2022/10/19 20:48:22 INFO mlflow.tensorflow: Validation succeeded!
Registered model 'Third Model' already exists. Creating a new version of this model...
2022/10/19 20:48:31 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Third Model, version 2
Created version '2' of model 'Third Model'.
